In [20]:
import pandas as pd
import os

# === Đường dẫn tới thư mục chứa file ===
folder_path = "Tổng hợp ngân hàng"

# === Đọc dữ liệu từ file Excel ===
kqkd_file = os.path.join(folder_path, "KQKD_TongHop.xlsx")
cdkt_file = os.path.join(folder_path, "CDKT_TongHop.xlsx")

df_kqkd = pd.read_excel(kqkd_file)
df_cdkt = pd.read_excel(cdkt_file)

# === TÍNH TÀI SẢN SINH LÃI ===
df_cdkt["Tài sản sinh lãi"] = (
    df_cdkt["III. Tiền, vàng gửi tại các TCTD khác và cho vay các TCTD khác"]
    + df_cdkt["VI. Cho vay khách hàng"]
    + df_cdkt["VIII. Chứng khoán đầu tư"]
)

# === TÍNH TÀI SẢN SINH LÃI BÌNH QUÂN ===
df_cdkt = df_cdkt.sort_values(by=["Bank", "Quý"]).copy()
df_cdkt["TS sinh lãi quý trước"] = df_cdkt.groupby("Bank")["Tài sản sinh lãi"].shift(1)
df_cdkt["TS sinh lãi bình quân"] = (
    df_cdkt["Tài sản sinh lãi"] + df_cdkt["TS sinh lãi quý trước"]
) / 2

# === GỘP DỮ LIỆU KQKD VỚI CDKT ===
df_merge = pd.merge(
    df_kqkd,
    df_cdkt[["Bank", "Quý", "TS sinh lãi bình quân"]],
    on=["Bank", "Quý"],
    how="inner"
)

# === TÍNH NIM (%) ===
df_merge["NIM (%)"] = (
    df_merge["I. Thu nhập lãi thuần"] / df_merge["TS sinh lãi bình quân"]
) * 100

# === CHỌN CÁC CỘT CẦN THIẾT ===
df_nim = df_merge[[
    "Bank",
    "Quý",
    "I. Thu nhập lãi thuần",
    "TS sinh lãi bình quân",
    "NIM (%)"
]]

# === LƯU KẾT QUẢ RA FILE EXCEL ===
output_file = os.path.join(folder_path, "NIM_theo_quy.xlsx")
df_nim.to_excel(output_file, index=False)

print("✅ Đã tính xong NIM (%) và lưu vào:", output_file)


✅ Đã tính xong NIM (%) và lưu vào: Tổng hợp ngân hàng\NIM_theo_quy.xlsx
